# Sentence correction gpt on Korean

한국어 문법교정 GPT3 모델 구축하기


Google Colab 환경에서 실행시 이 라인을 먼저 실행한다.

In [ ]:
# only google cloab
import os, sys
from google.colab import drive
drive.mount('/content/drive')

LOCATION = '/content/drive/MyDrive/spell_correction'

Local 환경에서 실행시 이 라인을 먼저 실행한다.

In [ ]:
LOCATION = '.'

In [ ]:
%pip install pandas numpy scipy scikit-learn torch accelerate transformers ipywidgets tqdm matplotlib

# Transformers 를 사용하기 위한 라이브러리들 설치

### AI-Hub data 정제

예시 데이터는 다음과 같다.

이 프로젝트를 수행하는 데이터인 AI-Hub 데이터는 용량이 112GB, 국외반출 불가로 데이터 관리에 주의를 요한다.

이 데이터를 Pandas.DataFrame['index', text', 'corrected'] 로 정제하는 과정을 거친다.

```json
{  "id": "100008-1-1-1",
  "fileName": "TX_CA_1_100008-1-1-1",
  "dataSet": "한국어 철자 및 맞춤법 교정용 병렬 데이터",
  "domain": "CA",
  "ko": "지금까지 다녀 본 여행지 중 좋았던 곳 추천해줘.",
  "corrected": "지금까지 다녀 본 여행지 중 좋았던 곳 추천해 줘.",
  "error": [
    {
      "errorType": "spac",
      "startPoint": 22,
      "endPoint": 27
    }
  ]
}
```

# 아래의 셀은 반드시 Local 환경에서 실행한다.

In [ ]:
from typing import List, Dict
from tqdm.notebook import tqdm

from json_processing import get_json_files, read_json_file

import pandas as pd

train = get_json_files(f'{LOCATION}/data/train')
validate = get_json_files(f'{LOCATION}/data/validate')

train_data = pd.DataFrame()
validate_data = pd.DataFrame()

In [ ]:
from json_processing import get_json_files, read_json_file
import multiprocessing as mp

# https://zerohertz.github.io/multiprocessing/
with mp.Pool() as pool:
    train_raw = list(tqdm(pool.imap(read_json_file, train), total=len(train)))

train_data = pd.concat(train_raw, ignore_index=True)

for file in tqdm(validate):
    validate_data = pd.concat([validate_data, read_json_file(f'{LOCATION}/{file}')], ignore_index=True)

for data in [train_data, validate_data]:
    data.dropna(subset=['text'], inplace=True)
    data['corrected'].fillna(data['text'], inplace=True)

train_data.to_csv(f'{LOCATION}/train.csv')
validate_data.to_csv(f'{LOCATION}/validate.csv')

### Transformer 학습 준비

학습 준비된 데이터는 토크나이저를 불러와 DataLoader에 준비하여 학습 준비를 한다.

파인튜닝 대상 모델은 다음과 같다.
[kykim/gpt3-kor-small_based_on_gpt2](https://github.com/kiyoungkim1/LMkor)

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast

import pandas as pd
import torch

LOCATION = '.'

# GPT-3 모델과 토크나이저 불러오기
tokenizer = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")
max_length = 100
loader_size = 16

# 훈련용 및 검증용 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = str(self.data.iloc[idx]['text'])
        output_text = str(self.data.iloc[idx]['corrected'])
        input_ids = self.tokenizer.encode(input_text, max_length=self.max_length, truncation=True, padding='max_length')
        output_ids = self.tokenizer.encode(output_text, max_length=self.max_length, truncation=True, padding='max_length')
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'output_ids': torch.tensor(output_ids, dtype=torch.long)
        }

train = pd.read_csv(f'{LOCATION}/train.csv')
validate = pd.read_csv(f'{LOCATION}/validate.csv')

train_dataset = TextDataset(train, tokenizer, max_length)
validate_dataset = TextDataset(validate, tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
validate_loader = DataLoader(validate_dataset, batch_size=32, shuffle=False)

In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

import torch
import pickle
import os

device = torch.device('cpu')

if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')

model = GPT2LMHeadModel.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2").to(device=device)

model.train()

lr = 2e-5
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
SAVE_PATH = f"{LOCATION}/all_metrics.pkl"

if os.path.exists(SAVE_PATH):
    with open(SAVE_PATH, "rb") as f:
        all_metrics = pickle.load(f)
else:
    all_metrics = []

def compute_metrics(pred):
    all_metrics.append({
        "train_loss": pred.metrics["train_loss"],
        "val_loss": pred.metrics["eval_loss"],
        "epoch": pred.metrics["epoch"],
        "f1": f1_score(pred.label_ids, pred.predictions.argmax(-1))
    })
    with open(SAVE_PATH, "wb") as f:
        pickle.dump(all_metrics, f)

def get_latest_checkpoint(path):
    checkpoints = [f for f in os.listdir(path) if f.startswith('checkpoint-')]
    if checkpoints:
        return os.path.join(path, max(checkpoints))
    else:
        return None

data_collactor = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=f"{LOCATION}/model", # 모델 저장 경로
    evaluation_strategy="epoch",  # 에포크마다 평가 수행
    learning_rate=2e-5,  # 학습률 설정
    num_train_epochs=50,  # 학습 에포크 설정
    per_device_train_batch_size=32,  # 배치 크기 설정
    warmup_steps=500,  # 워밍업 스텝 설정
    weight_decay=0.01,
    prediction_loss_only=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset,
    data_collator=data_collactor,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train(resume_from_checkpoint = True if get_latest_checkpoint(f'{LOCATION}/model/') else False)


trainer.save_model(f"{LOCATION}/model")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

with open(f"{LOCATION}/all_metrics.pkl", "rb") as f:
    all_metrics = pickle.load(f)

df_metrics = pd.DataFrame(all_metrics)

# 그래프 시각화
plt.plot(df_metrics["train_loss"], label="Train Loss")
plt.plot(df_metrics["validate_loss"], label="Validate Loss")
plt.legend()
plt.show()

plt.plot(df_metrics["f1"], label="F1 Score")
plt.legend()
plt.show()